# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
import os

file_path = '../output_data/cities.csv'

# Check if the file exists
if not os.path.isfile(file_path):
    # If not, create the file or handle it according to your use case
    with open(file_path, 'w') as file:
        # You can write initial data to the file if needed
        file.write('Header1,Header2\n')

# Now, proceed with accessing the file
city_data_df = pd.read_csv(file_path)

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,5.48,55,96,4.72,GS,1706748696
1,1,anadyr,64.7500,177.4833,-22.72,77,75,6.00,RU,1706748697
2,2,howrah,22.5892,88.3103,19.00,88,75,1.54,IN,1706748697
3,3,port moresby,-9.4431,147.1797,30.68,66,0,5.14,PG,1706748697
4,4,atar,20.5169,-13.0499,18.87,14,6,4.21,MR,1706748697


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,5.48,55,96,4.72,GS,1706748696
1,1,anadyr,64.7500,177.4833,-22.72,77,75,6.00,RU,1706748697
2,2,howrah,22.5892,88.3103,19.00,88,75,1.54,IN,1706748697
3,3,port moresby,-9.4431,147.1797,30.68,66,0,5.14,PG,1706748697
4,4,atar,20.5169,-13.0499,18.87,14,6,4.21,MR,1706748697


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display


# Configure the map plot
humidity_map = city_data_df.hvplot.scatter(
    x='Lng', y='Lat', c='Humidity', cmap='viridis',
    size='Humidity', alpha=0.7,
    title='City Humidity Map', height=500, width=700,
    colorbar=True, xlabel='Longitude', ylabel='Latitude'
)



# Display the map
humidity_map

:Scatter   [Lng]   (Lat,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df[(city_data_df['Max Temp'] > 25) & (city_data_df['Humidity'] > 40)]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,port moresby,-9.4431,147.1797,30.68,66,0,5.14,PG,1706748697
12,12,san antonio de pale,-1.4014,5.6325,28.05,78,100,4.83,GQ,1706748698
17,17,kuching,1.5500,110.3333,27.24,95,40,1.03,MY,1706748534
20,20,hawaiian paradise park,19.5933,-154.9731,26.69,70,75,2.57,US,1706748699
23,23,arteche,12.2645,125.4048,27.46,65,81,5.28,PH,1706748699


In [6]:
print(city_data_df[['Max Temp', 'Humidity']].describe())


         Max Temp    Humidity
count  553.000000  553.000000
mean    11.273237   74.909584
std     16.874288   19.947734
min    -39.000000   12.000000
25%      0.940000   65.000000
50%     17.290000   79.000000
75%     25.300000   90.000000
max     39.230000  100.000000


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,grytviken,GS,-54.2811,-36.5092,55,
1,anadyr,RU,64.7500,177.4833,77,
2,howrah,IN,22.5892,88.3103,88,
3,port moresby,PG,-9.4431,147.1797,66,
4,atar,MR,20.5169,-13.0499,14,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000  
params = {
    "type": "accommodation",
    "limit": 1,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{radius}@{lng},{lat}"
    params["bias"] = f"circle:{radius}@{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places" + geoapify_key



    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
grytviken - nearest hotel: No hotel found
anadyr - nearest hotel: No hotel found
howrah - nearest hotel: No hotel found
port moresby - nearest hotel: No hotel found
atar - nearest hotel: No hotel found
college - nearest hotel: No hotel found
vilyuchinsk - nearest hotel: No hotel found
parabel' - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
badger - nearest hotel: No hotel found
side - nearest hotel: No hotel found
san antonio de pale - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
khodzha-maston - nearest hotel: No hotel found
north branch - nearest hotel: No hotel found
kuching - nearest hotel: No hotel found
papatowai - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
blackmans bay - nearest hotel: No hotel found
jamestown - nearest hotel: No hotel f

,City,Country,Lat,Lng,Humidity,Hotel Name
0,grytviken,GS,-54.2811,-36.5092,55,No hotel found
1,anadyr,RU,64.7500,177.4833,77,No hotel found
2,howrah,IN,22.5892,88.3103,88,No hotel found
3,port moresby,PG,-9.4431,147.1797,66,No hotel found
4,atar,MR,20.5169,-13.0499,14,No hotel found
...,...,...,...,...,...,...
548,ceeldheer,SO,3.8488,47.1806,81,No hotel found
549,vallenar,CL,-28.5708,-70.7581,65,No hotel found
550,tremp,ES,42.1670,0.8949,61,No hotel found
551,scalea,IT,39.8130,15.7956,66,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE